# Tutorial 2a: *Running a generic single-point case*

This tutorial is an introduction to running single-point simulations of the Community Terrestrial Systems Model (CTSM) at locations that do not have preconfigured options.  It will guide you through setting up the required driver data (i.e. surface and atmosphere data) as well as setting up and submitting a single-point CTSM case.

In the previous tutorial, `Day1a_GlobalCase`, we set up and ran a global CTSM case. Many of the steps required to run a single-point case are similar, with some changes and additional required steps which we will cover here.
<br><br>

### Questions about this tutorial? 
- Please post them on the [CTSM forum in the CESM Bulletin Board](https://bb.cgd.ucar.edu/cesm/forums/ctsm-clm-mosart-rtm.134/). Note that this resource will require you to register and log in so that you can be notified of responses to your inquiries. 
- You can also file issues on the [NCAR CTSM-Tutorial GitHub repository](https://github.com/NCAR/CTSM-Tutorial-2022).
***

## In this tutorial

The tutorial has several components. Below you will find steps to: 
1. Generate subset surface and atmosphere data files at a single latitude and longitude point.
2. Set up and submit a single-point case.

Specifically, we will simulate the Harvard Forest site for one year using data extracted from global datasets that are available for CTSM. 

<div class="alert alert-block alert-warning">
<b>Note:</b> This tutorial assumes you have completed the previous tutorials! Content in the Day 0a Git Started tutorial is required, and Day 0b NEON and Day 1 global tutorials are strongly recommended. If you haven't completed the Day 0 and Day 1 tutorials, go back and do these first.
</div>

***

<div class="alert alert-block alert-info" markdown="1">

<b>TIP:</b>  Before we get started, make sure you're in a bash kernel 
<ul>
    <li> Switch kernel (upper right of your current notebook)</li>
    <li> Select either one of the Bash Kernels from the pop-up window</li>    
    <li> Click select</li>    
</ul>
</div>


<div>
<img src="https://github.com/NCAR/CTSM-Tutorial-2022/raw/main/images/kernel.png" width="670" />
</div>


***

<h1> 1. Subset global surface and atmosphere files </h1>

CTSM uses a surface data file to read in important grid cell-level information like vegetation, crop, and glacier grid cell fractions, the fractional cover of each plant functional type (PFT), and soil characteristics.

A global surface data file is located and read by default for global CTSM cases, depending on the chosen _component set_ and _resolution_. To run CTSM at a single point, we will need to supply a surface data file at a specified latitude and longitude.


<div class="alert alert-block alert-info">
Remember from Day 1 that a <b>component set</b>, or colloquially a "compset", specifies a configuration for your case, including the component models, time period of simulation, and model physics options. The <b>resolution</b> defines the model resolution or grid size.
</div>


Similarly, when running a land-only simulation, that is when using a "data atmosphere model" (e.g., in _DATM mode_), with climate data (e.g. temperature, precipitation, solar radiation, etc.) driven by an input file, CTSM needs DATM files. We can also provide subset global DATM for single-point runs.

<div class="alert alert-block alert-info">
<b>Tip:</b> It is not strictly required to provide CTSM/CIME with subset DATM data in order to run a single-point case, as CTSM can just use the global files. However, your simulations will run much faster if you use subset climate data.
</div>

<h2> 1.1 Use <em>subset_data</em> to subset surface and DATM files</h2>

We have created a python script, *subset_data*, which will subset default global surface and DATM files at a user-specified latitude and longitude.

This script is located in the CTSM source code, in the `tools/site_and_regional` folder.

Navigate here now:

In [ ]:
cd ~/CTSM/tools/site_and_regional
ls

In order to use the `subset_data` python script you must have some required python packages installed. On NCAR machines (like Cheyenne), you can load the NCAR python library, _ncar pylib_, by running `module load python` and then `ncar_pylib`. **This is not necessary in the cloud because your python environment is already configured in CESM-Lab.**

<div class="alert alert-block alert-info">
<b>Tip:</b> On NCAR or other machines, you can also use your own python environment if you want. Required third-party python packages are <i>scipy</i>, <i>xarray</i>, and <i>numpy</i>.
</div>

You can use the built-in print help to see what options are available for the subset data script:

In [ ]:
./subset_data --help

#### There are a lot of options, but for now we will just use a few of the most commonly used:

**Type of subsetting:**<br>
`point` : this tells the script to subset data at a single point (region is the other option)<br>

**Location-related information:**<br>
`--lat` : this tells the script which latitude to subset at (*must be between -90 and 90*)<br>
`--lon` : this tells the script which longitude to subset at (*can be between 0 and 360 or -180 and 180*)<br>
`--site` : optional, specifies a site name or tag<br>

**Type of files to create:**<br>
`--create-surface` : tells the script to subset surface data<br> 
`--create-datm` : tells the script to subset DATM data<br>

**Time information:** <br>
`--datm-syr` and `--datm-eyr`: starting and ending years for the DATM data to subset (*must be between 1901 and 2014*)<br>

**Data management information:**<br>
`--create-user-mods` : tells the script to create a *user_mods* directory (see below). Note that if you don't use this option, you will have to modify scripts in your simulation to point to the modifed files.<br> 
`--outdir` : specifies the directory to place subset data and user mods directory in<br>

#### Call the script with these options by running the line of code below 
A few notes:
- We will use a latitude of 42.53562 and longitude of 287.82438 (i.e. Harvard Forest). Note that your latitude and longitude points do not have to be this precise for your own sites!

- We will run the simulation from 2001 to 2002. Note that because we are only simulating years around present day, we are not using land use change. If you want to run a transient simulation, you will also need to create land use data (the create_landuse option).

- This is also pretty time consuming, so we'll use `qcmd_serial` here to put this in the queue on a single processor.  

- As before, please be patient while this runs, and don't worry if you see `WARNING: No dominant pft type is chosen.`

In [ ]:
qcmd_serial -- ./subset_data point --lat 42.53562 --lon 287.82438 --site my_point --create-surface --create-datm --datm-syr 2001 --datm-eyr 2002 --create-user-mods --outdir /scratch/$USER/my_subset_data

echo "------------------------"
echo "Successfully subset data"

Depending on the speed of your computing system, it may take a bit of time to subset all the climate data.

<h2> 1.2 Check on the subset files </h2>

Once the subsetting has successfully finished, let's navigate to the specified output directory to check on the data that we just created:

In [ ]:
cd /scratch/$USER/my_subset_data
ls

You should see a surface data file (e.g. *surfdata_0.9x1.25 ... .nc*) and two folders: **datmdata** and **user_mods**. 

* **datmdata** houses the subset DATM files
* **user_mods** is a directory created that houses several files we will use to set up our single-point case

Let's navigate into the **user_mods** directory to look at the contents:

In [ ]:
cd user_mods
ls

You should see three files: *shell_commands*, *user_nl_clm*, and *user_nl_datm_streams*.  

The *shell_commands* file contains *xmlchange* commands required to set up a single point case at the specified latitude and longitude.

Take a look at this file if you want:

In [ ]:
cat shell_commands

Note that many of the xml commands are changing aspects of the model configuration that are communicated to **[CIME](https://github.com/ESMCI/cime)** (Common Infrastructure for Modeling the Earth), which is the infrastructure that generates model executables and associated input files. Below are explanations of the commands included in this script. 

`./xmlchange CLM_USRDAT_DIR` - this tells CIME the location of an argument *CLM_USRDAT_DIR* which we can use to specify the main directory of subset data files  

`./xmlchange PTS_LON` and `./xmlchange PTS_LAT` - this tells CIME that we are running at a specified latitude and longitude  

`./xmlchange MPILIB` - this specifies a specific MPI (*Message Passing Interface*) library to use required for single-point runs on NCAR machines.  
 
   
If you remember from the Day 1 tutorial, *user_nl_clm* is a Fortran namelist file used to set up different namelist options for CLM. Here, we are using it to specify the location of our subset surface data. Note the use of the variable `$CLM_USRDAT_DIR` set up in the *shell_commands* file.
  
    
Similarly, *user_nl_datm_streams* specifies the location and a few other options for our subset DATM data.


We will use this **user_mods** directory when we create our single-point case (see below).

<div class="alert alert-block alert-warning">
<b>Note:</b> If for whatever reason you end up moving the subset data directory (i.e. here <b>/scratch/$USER/my_subset_data</b>), you will need to modify the xmlchange command that specifies the <i>CLM_USRDAT_DIR</i> to be the full path to the directory's new location. 
</div>

<h1> 2. Create a single-point CTSM case </h1>

Now that we have our subset data ready to go, we can set up our single-point case with CIME.

The steps required here are very similar to the global case that we set up in the Day 1 tutorial, with a few differences. Mainly, we are going to specify a `--user-mods-dir` in our `./create_newcase` command as the full path to the **user_mods** folder we just created with *subset_data* to point to all the data required to run the single point (rather than global) simulation. We will also choose a different component set and resolution.

<h2> 2.1. Create the case </h2>

As in our Day 1 Tutorial, we will navigate into the CTSM **scripts** directory to run the `create_newcase` script. Today we are going to be running a CLM-BGC simulation.

In [ ]:
cd ~/CTSM/cime/scripts
./create_newcase --case ~/clm_tutorial_cases/I2000_CTSM_singlept --res CLM_USRDAT --compset I2000Clm51BgcCrop --run-unsupported --user-mods-dirs /scratch/$USER/my_subset_data/user_mods

This command should look fairly familiar to you, with some updated values and arguments.

`--res` - defines the model resolution, or grid:
* we are now using `CLM_USRDAT`, which should be used when we have user-specified domain (i.e. a subset surface data)

`--compset` - defines the component set for the case:
* `I2000Clm51BgcCrop` is an alias that describes using year 2000 initialization time, data-driven atmosphere (GSWP3v1 data), CLM 5.1 BGC with prognostic crop, along with some other component settings.

`--user-mods-dirs` - this is where we tell CIME where our **user mods** directory is.
* it should be the path to the directory that was created during our *subset_data* scripting
* the namelist files (i.e. *user_nl_clm* and *user_nl_datm_streams*) will be copied into the case directory, and the commands within *shell_commands* will be executed. Remember that the information included in these files ensures the model uses the data subset for our site.

<div class="alert alert-block alert-info">
<b>Tip:</b> Depending on your machine (e.g. this is required on Cheyenne), you may also have to provide a project id (<code>--project {PROJECT_ID}</code>) which specifies accounting or directory permissions when on a batch system. By default the script uses the shell environment variable <code>$PROJECT</code>, which can be set in your bash profile

<i>Remember that you can see script parameters and definitions using</i> <code>./create_newcase --help</code>
</div>

<h2> 2.2. Set up the case and build the executable </h2>

As with our global case, we will change into our case directory, set up our case (`./case.setup`) and then build the executable with qcmd (`qcmd -- ./case.build`):

In [ ]:
cd ~/clm_tutorial_cases/I2000_CTSM_singlept
./case.setup
./xmlchange MPILIB=impi
qcmd -- ./case.build

<div class="alert alert-block alert-warning">
<b>Note:</b> You'll notice we changed the <code>MPILIB</code> parameter from what was set in the <b>user_mods</b>. This is a special change we have to make to run on this cloud system. On NCAR machines (e.g. Cheyenne) you should keep <code>MPILIB=mpi-serial</code>.
</div>


-----

You can read on, but before executing any code blocks in the notebook **wait for the model to build.**
This can take a while, especially while you're wating for your `qcmd` job to start and as code for the land model compiles.

You'll see text stating `MODEL BUILD HAS FINISHED SUCCESSFULLY` when it's finished.

<h2> 2.3. Customize the case </h2>

As in our global case, we will invoke a few XML commands to update some runtime values:

In [ ]:
./xmlchange STOP_OPTION=nyears
./xmlchange STOP_N=1
./xmlchange RUN_STARTDATE='2001-01-01'
./xmlchange DATM_YR_ALIGN=2001
./xmlchange DATM_YR_START=2001
./xmlchange DATM_YR_END=2002
./xmlchange PIO_REARRANGER_LND=2

<h3> Spinup </h3>

When running a model like CLM, the initial conditions (i.e. state variables like carbon and nitrogen pools and soil moisture) have an impact on the results of the simulation. Often, we don't know the precise values of these initial conditions. To get around this issue, we can initialize the model with arbitrary values and then run the model with some cycle of atmospheric forcing for many years (e.g. 200) until the model attains an equilibrium state. Then, we can simulate the model response to some perturbation (e.g. changing climate, CO<sub>2</sub>, etc.). This process -- establishing an equilibrium state -- is called _spinup_.

We have already run such a spinup simulation at Harvard Forest. We can see how the soil C pools evolved for this simulation over time.

*Note that you'll see **AD mode** for accelerated decomposition mode and **Post-AD mode** on this figure. See the tip below or visit the [Model Equilibrium and its Acceleration](https://escomp.github.io/ctsm-docs/versions/release-clm5.0/html/tech_note/Decomposition/CLM50_Tech_Note_Decomposition.html#model-equilibration-and-its-acceleration) section of the CLM Tech Note for more information*

<div>
<img src="https://github.com/NCAR/CTSM-Tutorial-2022/raw/main/images/ad_mode.png" width="525" height="375" alt="Evolution of different C pools during the accelerated decomposition spinup."/>
</div>

<div class="alert alert-block alert-info">
The steady-state, or equilibrium, size of carbon (C) and nitrogen (N) pools are proportional to their tunrover time. This spinup simulation was conducted using "accelerated decomposition", or "AD" mode. This accelerates the turnover time of "slow" ecosystem C and N pools (soil, wood, and coarse woody debris) so they come into equilibrium more quickly. 

We ran the model in AD mode for 100 years cycling through atmospheric forcing for 1981 to 2000.
AD mode was invoked with the commands: <code>./xmlchange CLM_FORCE_COLDSTART=on</code> and <code>./xmlchange CLM_ACCELERATED_SPNIUP=on</code>.

We then ran a "post-AD" simulation (using the end of our "AD" simulation as the starting point) for another 100 years with <code>./xmlchange CLM_ACCELERATED_SPINUP=off</code>. In returning the turnover times of slow C and N pools to their intentended rates, we have to adjust the pool sizes from their "AD" steady-state. For example if the turnover of "passive" soil C was 10x faster in AD mode, the passive soil C pool needs to be 10x larger starting the post-AD simulation (the model automatically handles this conversion for you). Then we ran the simulation for another 100 years to allow the state variables to equilibrate with non-accelerated decomposition.  In post-AD mode the history files are monthly, whereas AD output is set to annual averages by default.  This difference in history file output frequency is reflected in the variability in the post-AD GPP output.
</div>


After spin up is complete, we have to tell CIME to use the spinup simulation's end point as the starting point, or initial conditions, for our simulation. 

We do this via the <i>user_nl_clm</i> file:

In [ ]:
echo "finidat='/scratch/data/day2/finidat_file/I2000_CTSM51_spinup.clm2.r.0281-01-01-00000.nc'" >> user_nl_clm

<div class="alert alert-block alert-info">
<b>Tip:</b> You can also do this from the command line in a terminal window using any text editing software (e.g. vi, emacs, etc.)
</div>

Let's check the file to make sure our command worked:

In [ ]:
cat user_nl_clm

<h2> 2.4. Submit the case </h2>

Finally, let's submit the case as we did in Day 1:

In [ ]:
./case.submit

You should see a confirmation that it successfully submitted.

### Congratulations! You've created and submtted a single-point CLM case!

You can check the status of your case as in previous tutorials.

In [ ]:
qstat -u $USER

---
Once your jobs are complete (or show the 'C' state under the 'Use' column, which means complete), we can check the CaseStatus file to ensure there were no errors and it submitted, ran, and completed successfully.  To do this, we'll 'tail' the end of the CaseStatus file:

In [ ]:
tail ~/clm_tutorial_cases/I2000_CTSM_singlept/CaseStatus

Remember from Day1a that you can check on files in your runs in the **scratch/{USER}/{CASE_NAME}/run** directory (e.g. `/scratch/$USER/I2000_CTSM_singlept/run`). 

Archived history files will be in the **scratch/{USER}/archive/{CASE_NAME}/lnd/hist** directory (e.g. `/scratch/$USER/archive/I2000_CTSM_singlept/lnd/hist`).

Check on the status of your case to see that it's running as expected
____

Next, check out the next Generic Single Point Tutorial [2b_GenericSinglePoint_Visualization](Day2b_GenericSinglePoint_Visualization.ipynb) to walk through how to visualize and analyze some of the output produced. **Note that you don't need to wait for this job to finish before moving on to the Day2b tutorial**

In [ ]:
ls /scratch/$USER/I2000_CTSM_singlept/run

In [ ]:
cat /scratch/$USER/I2000_CTSM_singlept/run/lnd.log.288.220523-144323